In [36]:
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import math
from skimage import io
from skimage.segmentation import slic, quickshift, find_boundaries, felzenszwalb
from skimage import feature
from scipy import ndimage
from skimage import exposure
from skimage.filters import rank, sobel_h, sobel_v, sobel, roberts, threshold_otsu
from skimage.filters.rank import enhance_contrast, median, mean, mean_bilateral, otsu
from skimage import color
from skimage.morphology import disk
from scipy.ndimage import morphology
from scipy.ndimage import measurements
from skimage.exposure import equalize_hist
from skimage import feature
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from sklearn.cluster import MeanShift, estimate_bandwidth

%matplotlib inline

import os  
for fn in os.listdir("./train_images/"):
    # The input image
    solar = r"./train_images/" + fn
    
    # Convert image to numpy array
    img = io.imread(solar)

    imgGray = color.rgb2gray(img)
    equalized_image = equalize_hist(imgGray)
    #plt.imshow(equalized_image)
    #plt.show()
    #plt.hist(equalized_image)
    #plt.show() 

    equalized_image = median(equalized_image, disk(1))
    equalized_image = equalize_hist(equalized_image)
    edges1 = feature.canny(equalized_image)
    #plt.imshow(edges1)
    #plt.show()
    fillHoles1 = morphology.binary_fill_holes(edges1)
    #plt.imshow(fillHoles1)
    #plt.show()

    equalized_image = enhance_contrast(equalized_image, disk(1))
    equalized_image = equalize_hist(equalized_image)
    edges1 = feature.canny(equalized_image)
    #plt.imshow(edges1)
    #plt.show()
    fillHoles2 = morphology.binary_fill_holes(edges1)
    #plt.imshow(fillHoles2)
    #plt.show()

    equalized_image = median(equalized_image, disk(1))
    equalized_image = equalize_hist(equalized_image)
    edges1 = feature.canny(equalized_image)
    #plt.imshow(edges1)
    #plt.show()
    fillHoles3 = morphology.binary_fill_holes(edges1)
    #plt.imshow(fillHoles3)
    #plt.show()

    equalized_image = median(equalized_image, disk(1))
    equalized_image = equalize_hist(equalized_image)
    edges1 = feature.canny(equalized_image)
    #plt.imshow(edges1)
    #plt.show()
    fillHoles4 = morphology.binary_fill_holes(edges1)
    #plt.imshow(fillHoles4)
    #plt.show()

    img[:, :, 0] = img[:,:,0]*equalized_image
    img[:, :, 1] = img[:,:,1]*equalized_image
    img[:, :, 2] = img[:,:,2]*equalized_image
    img = equalize_hist(img)
    #plt.imshow(img)
    #plt.show()


    fillHoles5 = fillHoles3 + fillHoles1 + fillHoles2 + fillHoles4
    #plt.imshow(fillHoles5)
    #plt.show()
    img = io.imread(solar)
    img = equalize_hist(img)

    img[:, :, 0] = img[:,:,0]*equalized_image
    img[:, :, 1] = img[:,:,1]*equalized_image
    img[:, :, 2] = img[:,:,2]*equalized_image
    img = equalize_hist(img)

    flat_image=np.reshape(img, [-1, 3])

    #Estimate bandwidth
    bandwidth2 = estimate_bandwidth(flat_image, quantile=.3, n_samples=1000)
    #print(bandwidth2)
    ms = MeanShift(bandwidth2, bin_seeding=True, min_bin_freq = 50)
    ms.fit(flat_image)
    labels=ms.labels_
    #print(labels)

    print(np.unique(labels))
    # Plot image vs segmented image
    plt.figure(2, figsize=(10,10))
    plt.subplot(2, 1, 1)
    #plt.imshow(img)
    plt.axis('off')
    plt.subplot(2, 1, 2)
    #plt.imshow(np.reshape(labels, [101,101]))
    plt.axis('off')
    
       
    labels = np.reshape(labels, [101, 101])
    image_label_overlay = color.label2rgb(labels, image=img, image_alpha=1)
    n = np.unique(labels).shape[0]
    if n%2 == 0:
        fig, axs = plt.subplots(2,n/2 + 2, figsize=(15, 6))
    else:
        fig, axs = plt.subplots(2, int(math.ceil(n/2)) + 2, figsize=(15, 6))
    axs = axs.ravel()
    originalImage = io.imread(solar)
    axs[0].imshow(originalImage)
    axs[1].set_title("Original Image")
    axs[1].imshow(img)
    axs[1].set_title("Preprocessed Image")
    axs[2].imshow(image_label_overlay)
    axs[2].set_title("Regions Detected : " + str(n))
    
    i = 0
    for region in regionprops(labels):
        #print(region.area)
        # draw rectangle around segmented coins
        #minr, minc, maxr, maxc = region.bbox
        #rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
        #                          fill=False, edgecolor='red', linewidth=5)
        #ax3.add_patch(rect)
        originalImage[:, :, 0] = np.multiply(originalImage[:,:,0], (labels == i))
        originalImage[:, :, 1] = np.multiply(originalImage[:,:,1], (labels == i))
        originalImage[:, :, 2] = np.multiply(originalImage[:,:,2], (labels == i))
        axs[i+3].imshow(labels == i)
        axs[i+3].set_title("Region No : " + str(i+1))
        i += 1
    
    fig.savefig('./train_images/preprocess output/' + 'output1' + fn)
    plt.close('all')

[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1]
[0 1 2 3]
[0 1]
[0 1 2 3]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2 3]
[0 1 2 3]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2]
[0 1 2 3]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1 2]
[0 1 2]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1 2]
[0 1]
[0

IOError: [Errno 13] Permission denied: './train_images/intermediate preprocess'